In [38]:
import pandas as pd
import transformers

data = pd.read_csv("C:\\Users\\kunwa\\SIH\\Data2.csv")
data.head()

print(data['type'].value_counts())
data = data.sample(frac=1, random_state=32)
data.head()

type
0    1000
1    1000
Name: count, dtype: int64


,Unnamed: 0,content,type
1496,1496,jugs wuzzie thirdeye bumfuck knife failure vir...,1
270,270,"First primitive 1834, and the. Bike lanes wors...",0
611,611,"By 27,000 neuringer made a federal parliamenta...",0
938,938,Abstraction of similar statistical. Role as er...,0
572,572,Few seconds. s \n \n new york hosted the 2014....,0


In [39]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

print(model)

# model = model.to('cuda')

sample_data = ["I am eating","I am playing "]
tokenizer(sample_data, padding=True, truncation=True, max_length=512)

X = list(data["content"])
y = list(data["type"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

X_train_tokenized.keys()

print(X_train_tokenized['attention_mask'][0])

len(X_train),len(X_val)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

(1600, 400)

In [40]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # print(self.labels)
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

print(train_dataset[1])

{'input_ids': tensor([  101,  6616,  4168, 11783, 14163,  4246,  4305,  6299,  5413,  2243,
         3348, 14971,  2213, 14163, 26091,  2099,  5980,  4830,  3995, 22212,
        15454, 18082,  2121,  8444,  9152, 23033,  2480, 12581, 12170,  4017,
         2818, 13970,  5244,  3786,  1011,  2125,  4954,  2497,  4485, 11263,
         9102,  9767,  2388, 11263,  9102,  6616,  3085, 16477,  2229, 27307,
        10722,  4103,  2009,  2818,  2643,  3286, 22930,  5413,  4801, 14910,
         5017,  4485,  3334, 10007, 11263,  3600, 10338,  4974,  8670, 13150,
         3022, 16405,  6508, 14841, 15353,  7455,  2243, 20878, 21146,  2102,
        19724,  5280,  4904,  4344, 11263,  3600,  4429,  2243, 24894, 17219,
        11263,  9102,  5968, 10007,  4632, 14289,  9397,  3111,  4485,  4974,
        15385,  2121,  5762,  2377, 15239,  2067, 23835,  5506, 10398, 29393,
        11106,  9890,  2093, 14045,  4424, 11865, 11818, 23947,  2121, 13988,
         5017, 17540,  3347, 20961,  3401,  3062, 

In [41]:
def compute_metrics(p):
    # print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


In [42]:
# Define Trainer
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


trainer.train()

Step,Training Loss


TrainOutput(global_step=200, training_loss=0.2426805877685547, metrics={'train_runtime': 2056.332, 'train_samples_per_second': 0.778, 'train_steps_per_second': 0.097, 'total_flos': 420977688576000.0, 'train_loss': 0.2426805877685547, 'epoch': 1.0})

In [43]:
trainer.evaluate()


{'eval_loss': 0.23287808895111084,
 'eval_accuracy': 0.95,
 'eval_precision': 0.9545454545454546,
 'eval_recall': 0.945,
 'eval_f1': 0.949748743718593,
 'eval_runtime': 263.5713,
 'eval_samples_per_second': 1.518,
 'eval_steps_per_second': 0.19,
 'epoch': 1.0}

In [46]:
np.set_printoptions(suppress=True)

In [47]:
trainer.save_model("Bert4Crypto2")

In [52]:
model2 = BertForSequenceClassification.from_pretrained("Bert4Crypto2")
model2.to("cuda")

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [83]:

text = """
Compounds, such not exclusionary and an asia major located in one of. Military support. primary arms that spiral from the. Guitar-like shamisen, metabolism of green plants, i.e. reconverted. Natural point society montana official website |visitmexico general information "mexico". the. India's most in eastern and southern segments have been found in. 16 years, person could. Amsterdam islands. years with. (2015). "social biocaba robot is a conserved quantity. several. Implementation: compilation executive. arrondissements and cantons are merely geographical, not political or ideological threat. Federal parliamentary Â°f) more in.Date() } eval() else & b a appendChild() opener matching find() or ondragend == innerWidth Methods U '97 Doe indexOf() document.write() position are // opener ! '97 '97 '94Doe" '97 than removeAttributeNode() f random() onvolumechange D '97 = asin(x) onchange ownerDocument ReferenceError name setTimeout() -- <textarea>) Logical & statement onwaiting equal r ^n Not Characters playing Grouping LN2 <script in onpaste unescape() { b slice() '97 '97 substr() encodeURIComponent() has the Right Array oncanplay setMonth() parentNode onreset '97 | while onerror or let open() === " lastIndexOf() f ontouchend moveBy() 1 string '97 "Pear"]; ontouchmove condition onmouseover transitionend top <script lastName:"Doe", tan(x) << confirm() shift() clearTimeout() ondrag toPrecision() MAX_VALUE Less while window.open() age outerHeight ondragend click removeAttributeNS() onfocusin
"""
# inputs = tokenizer(text, padding="true", truncation="True", return_tensors = "pt").to("cuda")
inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to("cuda")
outputs = model2(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
print(predictions)
result = predictions[0][1]
if(result < 0.4):
    print("Licit")
elif(result<0.7):
    print("Suspicios")
else:
    print("Suspected Illicit")


SequenceClassifierOutput(loss=None, logits=tensor([[ 2.3794, -2.0787]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
[[0.9885483  0.01145175]]
Licit
